## Purpose of This Notebook

This notebook serves as an exploratory tool for examining the log file data from the horse behavioural experiments conducted in October and November 2023 that are loaded into a local DuckDB database using `logfile-to-database-CB.ipynb`.

It facilitates loading and querying the data from the database using some example SQL queries.



### Setup

In [ ]:
# | echo: false

from pathlib import Path
from pprint import pprint

import duckdb
import pandas as pd
from IPython.display import Markdown, display
from loguru import logger

from horse_logic.utils import set_custom_logger_format

In [ ]:
# | echo: false

# Display dataframes in a more friendly paginated manner

import itables.options as opt
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)
opt.pageLength = 20

In [ ]:
# | echo: false

set_custom_logger_format()

### Database information

In [ ]:
# | echo: false

DATA_DIR = Path("../data")
EXPERIMENT_TYPE = "CB"

assert DATA_DIR.exists()

DATA_DB = DATA_DIR / f"Experiments_{EXPERIMENT_TYPE}_2023-Q4.ddb"  # DuckDB database name
db_exists = DATA_DB.exists()

logger.info(f"Database file: {DATA_DB.resolve()}")

### Connect to database

In [ ]:
# | echo: false


def connect_ddb(database=DATA_DB):
    try:
        if db_exists:
            con = duckdb.connect(database=str(database))
            logger.info(f"CONNECTED - Database file: {database.resolve()}")
            return con
        else:
            logger.error(f"Database file: {database.resolve()} not found.")
    except Exception as e:
        print(
            f"Error with Database file: {database.resolve()} - Is database already open - check for .wal lock file?"
        )
        logger.error(f"{e}")
    return None


con = connect_ddb()

### Perform some example database queries (local DuckDB) 

Show the tables in the database - should be `EventCBs`, `ExperimentCBs`, `ResponseCBs` and `TrialCBs`.

In [ ]:
# Cross-check queries

con.sql("SHOW TABLES;")

### Show the distinct `EventTypes` in alphabetical order

In [ ]:
con.sql("SELECT DISTINCT EventType FROM EventCBs ORDER BY EventType")

### List all of the Experiments

In [ ]:
experiments_df = con.sql("SELECT * FROM ExperimentCBs").df()

In [ ]:
experiments_df;

### List of all the log filenames

In [ ]:
filenames_df = con.sql("SELECT LogFilename from ExperimentCBs ORDER By LogFilename").df()

In [ ]:
filenames_df.iloc[0, 0]

### Reconstruct the trial/event/response details for a particular experiment (via log filename)

In [ ]:
def display_file_contents(file_path):
    try:
        with open(file_path, "r") as file:
            contents = file.read()
            print(contents)
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except IOError:
        print(f"Error: There was an issue reading the file '{file_path}'.")

In [ ]:
example_log = DATA_DIR / "results" / "zips" / "cb_data" / filenames_df.iloc[0, 0]

In [ ]:
def sql_recreate_logfile(filename):
    return f"""
        WITH log_events AS (
        SELECT 
            e.EventTime,
            e.EventType,
            t.TrialNumber,
            t.ResponseType,
            t.TrialSubType,
            r.ResponseTime
        FROM EventCBs e
        LEFT JOIN TrialCBs t ON e.TrialID = t.TrialID
        LEFT JOIN ResponseCBs r ON e.TrialID = r.TrialID
        JOIN ExperimentCBs ex ON t.ExperimentID = ex.ExperimentID
        WHERE ex.LogFileName = '{filename}'
        ORDER BY e.EventTime
        )
        SELECT 
        strftime('%Y-%m-%d %H:%M:%S.%f', EventTime) || ': ' ||
        CASE 
            WHEN EventType LIKE 'GO for Trial%' OR EventType LIKE 'NOGO for Trial%'
            THEN EventType || 
                CASE 
                    WHEN ResponseTime IS NOT NULL THEN ' - response time ' || CAST(ResponseTime AS TEXT)
                    ELSE ''
                END
            ELSE EventType
        END AS {sanitize_filename_for_sql(filename)}
        FROM log_events;
    """

In [ ]:
example_log.name

In [ ]:
import re

def sanitize_filename_for_sql(filename):
    # Remove the file extension
    name_without_ext = filename.rsplit('.', 1)[0]
    
    # Replace spaces and hyphens with underscores
    name_underscored = name_without_ext.replace(' ', '_').replace('-', '_')
    
    # Remove any characters that are not alphanumeric or underscore
    sanitized = re.sub(r'[^\w]', '', name_underscored)
    
    # Ensure the name starts with a letter or underscore
    if not sanitized[0].isalpha() and sanitized[0] != '_':
        sanitized = '_' + sanitized
    
    # Truncate to a reasonable length if needed (e.g., 63 characters)
    sanitized = sanitized[:63]
    
    return sanitized

In [ ]:
sql_recreate_logfile(example_log.name)

In [ ]:
con.sql(sql_recreate_logfile(example_log.name))

In [ ]:
data_dir = example_log.parent

In [ ]:
example_log

In [ ]:
# Compare the actual file contents

display_file_contents(example_log)

#### Deep dives: 

#### `Experiment_2023-10-09T17:45:46.540359_apollo_5_Training randomised Type 1.log`

In [ ]:
deep_dive_filename = "Experiment_2023-10-09T17:45:46.540359_apollo_5_Training randomised Type 1.log"

In [ ]:
def get_experiment_details(filename):
    query = f"""
        SELECT
            ExperimentID, 
            Cohort,
            SubjectName, 
            SubjectNumber, 
            SessionNumber,
            ExperimentType, 
            Comment, 
            DateTime, 
            LogFileName
        FROM ExperimentCBs 
        WHERE LogFileName = '{filename}';
    """
    return con.sql(query).df()

In [ ]:
def get_trials_details(filename):
    query = f"""
        SELECT t.*
        FROM TrialCBs t
        JOIN ExperimentCBs e ON t.ExperimentID = e.ExperimentID
        WHERE e.LogFileName = '{filename}';
    """
    return con.sql(query).df()

In [ ]:
def get_events_details(filename):
    query = f"""
        SELECT ev.*
        FROM EventCBs ev
        JOIN TrialCBs t ON ev.TrialID = t.TrialID
        JOIN ExperimentCBs e ON t.ExperimentID = e.ExperimentID
        WHERE e.LogFileName = '{filename}';
    """
    return con.sql(query).df()

In [ ]:
def get_responses_details(filename):
    query = f"""
        SELECT r.*,
        t.TrialNumber,
        t.ResponseType,
        t.TrialSubType,
        t.Direction,
        t.CriterionType,
        t.CriterionCount,
        t.TrialStartTime,
        t.TrialEndTime,
        t.SessionType
        FROM ResponseCBs r
        JOIN TrialCBs t ON r.TrialID = t.TrialID
        JOIN ExperimentCBs e ON t.ExperimentID = e.ExperimentID
        WHERE e.LogFileName = '{filename}';
    """
    return con.sql(query).df()

In [ ]:
def compare_experiment_data(filename):
    display(Markdown(f"### Compare: {filename}"))
    display(get_experiment_details(filename))
    display(get_trials_details(filename))
    display(get_events_details(filename))  
    display(get_responses_details(filename))
    display(Markdown("### Original logfile\n"))
    display_file_contents(data_dir / filename)
    

In [ ]:
compare_experiment_data(deep_dive_filename)

In [ ]:
compare_experiment_data("Experiment_2023-10-11T10:26:14.965471_atom_8_Test Type 1.log")

### Close the connection to the database

To avoid file lock errors.

In [ ]:
# con.close()


### Potential prototype queries
```
SELECT AVG(r.ResponseTime) as AvgResponseTime
FROM Trial t
JOIN Response r ON t.TrialID = r.TrialID
WHERE t.ExperimentID = 1 AND t.ResponseType = 'GO';
```
```
SELECT TrialSubType, COUNT(*) as SuccessfulTrials
FROM Trial
WHERE CriterionType = 'positive'
GROUP BY TrialSubType;
```
Experiment duration

```
SELECT 
    e.ExperimentID,
    MIN(ev.EventTime) as StartTime,
    MAX(ev.EventTime) as EndTime,
    TIMESTAMPDIFF(SECOND, MIN(ev.EventTime), MAX(ev.EventTime)) as DurationSeconds
FROM Experiment e
JOIN Trial t ON e.ExperimentID = t.ExperimentID
JOIN Event ev ON t.TrialID = ev.TrialID
GROUP BY e.ExperimentID;
```